# Imports

In [1]:
from architectures.lenet5 import LeNet5
from architectures.alexnet import AlexNet
from architectures.vgg16 import VGG16
from architectures.resnet50 import ResNet50
from Datasets import load_cifar10, load_cifar100, load_mnist, load_fashion_mnist
from sklearn.model_selection import KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import pickle

# Load Datasets

In [2]:
datasets = {"cifar10": load_cifar10()#,
            #"cifar100": load_cifar100(),
            #"mnist": load_mnist(),
            #"fashion_mnist": load_fashion_mnist()
            }
            

# Load Architectures

In [3]:
architectures = {#"LeNet5": LeNet5,
                 "AlexNet": AlexNet#,
                 #"VGG16": VGG16,
                 #"ResNet50": ResNet50
                 }

# Set Hyperparameters

In [4]:
learning_rates = [(x ** 2) / 10000 if x != 0 else 0.0001 for x in range(0, 15, 3)]
print("Learning rates: " + str(learning_rates))

hyperparameters = {"optimizers": [keras.optimizers.SGD, keras.optimizers.Adam],
                   "learning_rates": learning_rates,
                   "epochs": 1,
                   "batch_size": 16
                   }

Learning rates: [0.0001, 0.0009, 0.0036, 0.0081, 0.0144]


# Train Models

In [5]:
# Iterate through all datasets
for dataset_name, dataset in datasets.items():
    X_train, Y_train, X_test, Y_test = dataset
    
    classes = Y_train.shape[1]
    filters = X_train.shape[3]
    
    # For testing
    X_train, _, Y_train, _ = train_test_split(X_train, Y_train, train_size=1024, test_size=1)
    
    
    # Iterate through all architectures
    for architecture_name, architecture in architectures.items():
        print(architecture_name, dataset_name)
        best_model = [None, None, 0.0]
        
        # Prepare data for cross validation
        kf = KFold(
        n_splits=len(hyperparameters["optimizers"]) * len(hyperparameters["learning_rates"])) 
        kf.get_n_splits(X_train)

        i = 0 # Index for optimizers
        j = 0 # Index for learning rates
        
        # Iterate through all combinations of learning rates and optimizers
        for train_index, test_index in kf.split(X_train):
            optimizer = hyperparameters["optimizers"][i]
            learning_rate = hyperparameters["learning_rates"][j]

            # Build and compile model
            arch = architecture(classes = classes,
                                optimizer = optimizer(learning_rate=learning_rate),
                                filters = filters)
            model = arch.model

            # Resize images
            X_tr = architecture.resize_images(X_train[train_index])
            Y_tr = Y_train[train_index]
            X_vl = architecture.resize_images(X_train[test_index])
            Y_vl = Y_train[test_index]

            # Generate edited images
            datagen = ImageDataGenerator(width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            fill_mode='nearest',
                            horizontal_flip = True)
            datagen.fit(X_tr)
            
            print(X_tr[0])
            
            # Fit model
            history = model.fit(datagen.flow(X_tr, Y_tr, batch_size=hyperparameters["batch_size"]),
                                  validation_data=(X_vl, Y_vl),
                                  epochs=hyperparameters["epochs"], steps_per_epoch=len(X_tr)/hyperparameters["batch_size"])
            
            val_accuracy = history.history['val_accuracy'][-1]
            
            # Check whether the new model is better
            if val_accuracy > best_model[2]:
                best_model = [model, history, val_accuracy]

            # Update indexes
            i = (i + 1) % len(hyperparameters["optimizers"])
            j = (j + 1) % len(hyperparameters["learning_rates"])
            
        # Evaluate best model
        _, test_accuracy = best_model[0].evaluate(architecture.resize_images(X_test), Y_test)
        
        # Save best model
        name = str("{:.2f}".format(test_accuracy) + "Acc_" +
                       architecture_name + "_" + dataset_name)
        
        best_model[0].save(name)

        hist_df = pd.DataFrame(best_model[1].history) 

        # save to json:  
        hist_json_file = name + '_history' + '.json' 
        with open(hist_json_file, mode='w') as f:
            hist_df.to_json(f)


AlexNet cifar10
[[[0.25525698 0.28461155 0.30758467]
  [0.31833208 0.35494024 0.38359016]
  [0.38140717 0.42526898 0.45959562]
  ...
  [0.36424458 0.42526984 0.4615036 ]
  [0.30400822 0.35494155 0.3851832 ]
  [0.24377106 0.28461227 0.30886176]]

 [[0.31833208 0.35494027 0.38359016]
  [0.39699328 0.4426475  0.47837692]
  [0.47565448 0.53035474 0.5731637 ]
  ...
  [0.45425096 0.5303558  0.5755431 ]
  [0.37912995 0.4426491  0.48036358]
  [0.30400792 0.3549412  0.3851828 ]]

 [[0.38140717 0.42526898 0.45959565]
  [0.47565448 0.53035474 0.5731637 ]
  [0.5699018  0.63544047 0.6867317 ]
  ...
  [0.54425734 0.6354418  0.6895826 ]
  [0.45425168 0.53035665 0.575544  ]
  [0.3642448  0.42527008 0.46150386]]

 ...

 [[0.45959657 0.47675994 0.47866693]
  [0.5731648  0.5945693  0.59694755]
  [0.68673307 0.7123787  0.7152282 ]
  ...
  [0.03704377 0.05699041 0.03989329]
  [0.03091772 0.04756572 0.033296  ]
  [0.02479158 0.03814089 0.02669862]]

 [[0.38359153 0.39791653 0.39950818]
  [0.47837862 0.49624

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=64d23a20-b5bc-41cc-930c-3bd7ede12b0c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>